In [1]:
from javascript import require, On, Once, AsyncTask, once, off
import math
from actions import Actions

In [2]:
import os

# Set the timeout to 100,000 milliseconds
os.environ['REQ_TIMEOUT'] = '10000000'

# Or disable the timeout
os.environ['REQ_TIMEOUT'] = '0'


In [3]:
# Import the javascript libraries
mineflayer = require("mineflayer")
pathfinder = require('mineflayer-pathfinder')
collectBlockPlugin = require('mineflayer-collectblock')
blockFinderPlugin = require('mineflayer-blockfinder')
vec3 = require("vec3")
# Global bot parameters
serverHost = "localhost"
serverPort = 3000
reconnect = True
REQ_TIMEOUT = 0

In [4]:
class Agent:

    def __init__(self, playerUsername, botName, serverHost, serverPort, reconnect=True):
        self.playerUsername = playerUsername
        self.botName = botName
        self.serverHost = serverHost
        self.serverPort = serverPort
        self.reconnect = reconnect
        self.bot = None
        self.botArgs = {
            "host": serverHost,
            "port": serverPort,
            "username": botName,
            "hideErrors": False,
        }
        
        self.createBot()
        

    def createBot(self):
        self.bot = mineflayer.createBot(self.botArgs)
        self.bot.loadPlugin(pathfinder.pathfinder)
        self.bot.loadPlugin(collectBlockPlugin.plugin)
        self.bot.loadPlugin(blockFinderPlugin)
        self.mcData = require('minecraft-data')(self.bot.version)
        self.movements = pathfinder.Movements(self.bot, self.mcData)
        self.bot.pathfinder.setMovements(self.movements)
        self.actions = Actions(self.bot, self.mcData, self.movements,self.playerUsername)
        self.startEvents()
        return self.bot
    
    def log(self, message):
        print(f"[{self.bot.username}] {message}")

    def startEvents(self):
        bot = self.bot
        botName = self.botName
        reconnectState = {"reconnect": self.reconnect}

        @On(bot, "login")
        def handleLogin(this):
            botSocket = bot._client.socket
            print(f"[{botName}] Logged in to {botSocket.server if botSocket.server else botSocket._host }")

        @On(bot, "kicked")
        def handleKicked(this, reason, loggedIn):
            if loggedIn:
                print(f"[{botName}] Kicked whilst trying to connect: {reason}")

        @On(bot, "messagestr")
        def handleMessagestr(this, message, messagePosition, jsonMsg, sender, verified=None):
            if messagePosition == "chat" and "quit" in message:
                reconnectState["reconnect"] = False
                this.quit()
            elif messagePosition == "chat" and'come' in message:
                localPlayers = bot.players
                for player in localPlayers:
                        playerData = localPlayers[player]
                        if playerData["uuid"] == sender:
                            target = localPlayers[player].entity
                            
                if not target:
                    bot.chat("I don't see you !")
                    return
                pos = target.position
                bot.pathfinder.setMovements(self.movements)
                bot.pathfinder.setGoal(pathfinder.goals.GoalNear(pos.x, pos.y, pos.z, 1))
            

        @On(bot, "end")
        def handleEnd(this, reason):
            print(f"[{botName}] Disconnected: {reason}")
            # Turn off old events
            off(bot, "login", handleLogin)
            off(bot, "kicked", handleKicked)
            off(bot, "messagestr", handleMessagestr)

            # Reconnect if the reconnect flag is set to True
            if reconnectState["reconnect"]:
                print(f"[{botName}] Attempting to reconnect")
                self.createBot()

            # Last event listener
            off(bot, "end", handleEnd)
        


  



In [5]:
# agent = Agent('jessica030327',"Johnny", "localhost", 3000)
# agent1 = Agent('jessica030327',"Cassie", "localhost", 3000)
agent2 = Agent('jessica030327',"Jon", "localhost", 3000)


[Jon] Logged in to localhost


In [6]:
agent2.actions.mineBlock('spruce_fence',6)

[JSE] Mineflayer detected that you are using a deprecated event (physicTick)! Please use this event (physicsTick) instead.

[JSE] Mineflayer detected that you are using a deprecated event (physicTick)! Please use this event (physicsTick) instead.

[JSE] Mineflayer detected that you are using a deprecated event (physicTick)! Please use this event (physicsTick) instead.

[JSE] Mineflayer detected that you are using a deprecated event (physicTick)! Please use this event (physicsTick) instead.

[JSE] Mineflayer detected that you are using a deprecated event (physicTick)! Please use this event (physicsTick) instead.

[JSE] Mineflayer detected that you are using a deprecated event (physicTick)! Please use this event (physicsTick) instead.

[JSE] c:\Users\junyu\anaconda3\envs\Minecraft_env\Lib\site-packages\javascript\js\node_modules\mineflayer\lib\plugins\inventory.js:177

[JSE]     await bot.lookAt(block.position.offset(0.5, 0.5, 0.5), false)

[JSE]                                     ^

[J

In [6]:
agent2.bot.inventory.items()

[
  Item {
    type: 109,
    count: 22,
    metadata: 0,
    nbt: null,
    stackId: null,
    name: 'oak_log',
    displayName: 'Oak Log',
    stackSize: 64,
    maxDurability: undefined,
    slot: 36
  }
]

In [7]:
chestPosition = vec3(-210,67, 530)
# agent2.actions.moveToChest(chestPosition)
agent2.actions.pathfinding(chestPosition,findPlayer = False)

In [5]:
chestPosition = vec3(-212,68, 451)
agent2.actions.placeItem( 'coarse_dirt', chestPosition,"up")


☕  JavaScript Error  Call to 'placeBlock' failed:
> agent2.actions.placeItem( 'coarse_dirt', chestPosition,"up")
  at <module> (C:\Users\junyu\AppData\Local\Temp\ipykernel_112104\3331974878.py:2)
> self.bot.placeBlock(referenceBlock, faceVector)
  at placeItem (j:\multiagentCollaboration\actions.py:171)
> 

... across the bridge ...

  at async Bridge.onMessage (c:\Users\junyu\anaconda3\envs\Minecraft_env\Lib\site-packages\javascript\js\bridge.js:231:7)
  at async Bridge.call (c:\Users\junyu\anaconda3\envs\Minecraft_env\Lib\site-packages\javascript\js\bridge.js:136:17)
  at async EventEmitter.placeBlock (c:\Users\junyu\anaconda3\envs\Minecraft_env\Lib\site-packages\javascript\js\node_modules\mineflayer\lib\plugins\place_block.js:33:5)
  at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
  at placeBlockWithOptions (c:\Users\junyu\anaconda3\envs\Minecraft_env\Lib\site-packages\javascript\js\node_modules\mineflayer\lib\plugins\place_block.js:13:36)
  at onceWith

In [15]:
chestPosition = vec3(-210,67, 489)
agent2.actions.moveToChest(chestPosition)

[Jon] No chest at Vec3 { x: -210, y: 67, z: 489 }, it is air


In [13]:
goalPosition = vec3(-260, 37, 450)
agent2.actions.pathfinding(goalPosition,findPlayer = True)
# agent1.pathfinding(goalPosition,findPlayer = False)
# agent.pathfinding(goalPosition,findPlayer = False)

[Jon] Disconnected: socketClosed
[Jon] Attempting to reconnect


In [6]:
# agent2.left(10)
# agent2.right(30)
# agent2.forward(5)
agent2.actions.forward(1)

In [7]:
mcData = require('minecraft-data')(agent2.bot.version)
mcData.itemsByName

{
  air: { id: 0, name: 'air', displayName: 'Air', stackSize: 64 },
  stone: { id: 1, name: 'stone', displayName: 'Stone', stackSize: 64 },
  granite: { id: 2, name: 'granite', displayName: 'Granite', stackSize: 64 },
  polished_granite: {
    id: 3,
    name: 'polished_granite',
    displayName: 'Polished Granite',
    stackSize: 64
  },
  diorite: { id: 4, name: 'diorite', displayName: 'Diorite', stackSize: 64 },
  polished_diorite: {
    id: 5,
    name: 'polished_diorite',
    displayName: 'Polished Diorite',
    stackSize: 64
  },
  andesite: { id: 6, name: 'andesite', displayName: 'Andesite', stackSize: 64 },
  polished_andesite: {
    id: 7,
    name: 'polished_andesite',
    displayName: 'Polished Andesite',
    stackSize: 64
  },
  deepslate: { id: 8, name: 'deepslate', displayName: 'Deepslate', stackSize: 64 },
  cobbled_deepslate: {
    id: 9,
    name: 'cobbled_deepslate',
    displayName: 'Cobbled Deepslate',
    stackSize: 64
  },
  polished_deepslate: {
    id: 10,
    n

In [ ]:
async function craftItem(bot, name, count = 1) {
    const item = mcData.itemsByName[name];
    const craftingTable = bot.findBlock({
        matching: mcData.blocksByName.crafting_table.id,
        maxDistance: 32,
    });
    await bot.pathfinder.goto(
        new GoalLookAtBlock(craftingTable.position, bot.world)
    );
    const recipe = bot.recipesFor(item.id, null, 1, craftingTable)[0];
    await bot.craft(recipe, count, craftingTable);
}

In [8]:
item = agent2.bot.registry.itemsByName['oak_planks']
craftingTableID = agent2.bot.registry.blocksByName.crafting_table.id
item = mcData.blocksByName['oak_planks']
item

{
  id: 13,
  name: 'oak_planks',
  displayName: 'Oak Planks',
  hardness: 2,
  resistance: 3,
  stackSize: 64,
  diggable: true,
  material: 'mineable/axe',
  transparent: false,
  emitLight: 0,
  filterLight: 15,
  defaultState: 15,
  minStateId: 15,
  maxStateId: 15,
  states: [],
  drops: [ 23 ],
  boundingBox: 'block',
  shapes: [ [ 0, 0, 0, 1, 1, 1 ] ]
}

In [9]:
item = mcData.blocksByName['oak_planks']
craftingTable = agent2.bot.findBlocks({
            "matching": craftingTableID,
            "maxDistance": 32,

        })
craftingTable

[ Vec3 { x: -35, y: 79, z: -49 } ]

In [10]:
# item = mcData.blocksByName['oak_planks']
item = agent2.bot.registry.itemsByName['oak_planks']
craftingTable = agent2.bot.findBlocks({
            "matching": craftingTableID,
            "maxDistance": 32,

        })

agent2.actions.pathfinding(craftingTable[0])

craftingTableBlock = agent2.bot.blockAt(craftingTable[0])
# print(craftingTableBlock)
recipe = agent2.bot.recipesFor(item.id, None, 1, craftingTable )
print(recipe)
agent2.bot.craft(recipe, 1, craftingTable,timeout = 1000 )

[
  Recipe {
    result: RecipeItem { id: 23, metadata: undefined, count: 4 },
    inShape: null,
    outShape: null,
    ingredients: [ [RecipeItem] ],
    delta: [ [RecipeItem], [RecipeItem] ],
    requiresTable: false
  }
]


In [11]:
agent2.bot.lookAt(craftingTableBlock.position.offset(0.5, 0.5, 0.5), False)

In [10]:
agent2.bot.craft(recipe, 1, craftingTable)

[JSE] c:\Users\junyu\anaconda3\envs\Minecraft_env\Lib\site-packages\javascript\js\node_modules\mineflayer\lib\plugins\inventory.js:177

[JSE]     await bot.lookAt(block.position.offset(0.5, 0.5, 0.5), false)

[JSE]                                     ^

[JSE] 

[JSE] TypeError: Cannot read properties of undefined (reading 'offset')

[JSE]     at EventEmitter.activateBlock (c:\Users\junyu\anaconda3\envs\Minecraft_env\Lib\site-packages\javascript\js\node_modules\mineflayer\lib\plugins\inventory.js:177:37)

[JSE]     at craftOnce (c:\Users\junyu\anaconda3\envs\Minecraft_env\Lib\site-packages\javascript\js\node_modules\mineflayer\lib\plugins\craft.js:39:13)

[JSE]     at EventEmitter.craft (c:\Users\junyu\anaconda3\envs\Minecraft_env\Lib\site-packages\javascript\js\node_modules\mineflayer\lib\plugins\craft.js:20:15)

[JSE]     at Bridge.call (c:\Users\junyu\anaconda3\envs\Minecraft_env\Lib\site-packages\javascript\js\bridge.js:136:42)

[JSE]     at Bridge.onMessage (c:\Users\junyu\anaconda

Exception: Call to 'craft' timed out. Increase the timeout by setting the `timeout` keyword argument.

13

In [12]:
# collectBlockPlugin = require('mineflayer-collectblock')
agent2.bot.loadPlugin(collectBlockPlugin.plugin)

In [25]:
targets = [agent2.bot.blockAt(block) for block in blocks]
agent2.bot.collectBlock.collect(targets, {
            'count': 3,
        })


Exception: Call to 'collect' timed out. Increase the timeout by setting the `timeout` keyword argument.

In [14]:
blockByName['id']

9

In [19]:
def mineBlock(self,name,count=1):
    blockByName = self.mcData.blocksByName[name]
    blocks = self.bot.findBlocks({
                "matching": blockByName.id,
                "maxDistance": 32,
                "count": count,
                'timeout': 30,
            })
    targets = [self.bot.blockAt(block) for block in blocks]
    self.bot.collectBlock.collect(targets, {
                'count': 3,
            })
    self.bot.chat((f"I have finished mining {name}"))
    